In [1]:
from statsbombpy import sb
import pandas as pd

I de første celler er Thors kode fra "passing seqeunces creation.ipynb" kopieret.
Jeg laver en ny markdown celle, når koden begynder at adskille sig.

In [2]:
#find ids for barca home matches - only home games because then locations can be compared between different games
matches = sb.matches(competition_id=11, season_id=90)
barca_home_matches = matches[matches["home_team"]=="Barcelona"]
match_ids = barca_home_matches['match_id'].values.tolist()
#All events sorted for barca home games and possession 
events = sb.competition_events(
    country="Spain",
    division= "La Liga",
    season="2020/2021",
    gender="male"
)
events = events[events['match_id'].isin(match_ids)]
df = events[events["team"]=="Barcelona"]
df = df[df["possession_team"]=="Barcelona"]
#filter threshold for Xg:
df_xg = df[~df['shot_statsbomb_xg'].between(0, 0.05)]
#Events sorted in a specific order so each passing sequence is correctly sorted
sequences_sorted = df_xg.sort_values(['match_id', 'period','timestamp'], ascending=[True, True, True])
#make new ids because right now there is ids from 1 to x for each match but it repeats from 1 and up in every match so each possession id points to different matches 
# - i just put the possession id after match_id in the newly created id
sequences_sorted['possession_id'] = sequences_sorted['match_id'].astype(str) + sequences_sorted['possession'].astype(str)
sequences_sorted['possession_id'] = sequences_sorted['possession_id'].astype(int)
#get the ids of sequences which contain a shot (contain an xg value)
shot_sequences = sequences_sorted[sequences_sorted["shot_statsbomb_xg"].notna()]
shot_sequences_ids = shot_sequences["possession_id"].unique()
#filter for possession sequences which end with a shot
sequences_filtered = sequences_sorted[sequences_sorted['possession_id'].isin(shot_sequences_ids)]
#fill all rows with an xg for the corresponding sequence - right now there are many missing values in "shot_statsbomb_xg"
sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
#now we dont need the shot event rows any more so remove them
sequences_filtered = sequences_filtered[sequences_filtered["type"]!="Shot"]
#filter the df to only include row with an id the of a pass recipient and we subset the columns
player_final_sequences =  sequences_filtered[sequences_filtered["pass_recipient"].notna()][["player_id", "pass_recipient_id", "possession_id", "xg","timestamp"]]
player_final_sequences
player_final_sequences = player_final_sequences.sort_values(['possession_id', 'timestamp'], ascending=[True, True])
player_final_sequences = player_final_sequences[player_final_sequences['player_id'] != player_final_sequences['pass_recipient_id']]
player_final_sequences['sequence'] = player_final_sequences.groupby('possession_id').cumcount(ascending=False) + 1
player_final_sequences
#remove sequences with few passes if wanted
index_counts = player_final_sequences['possession_id'].value_counts()
player_final_sequences = player_final_sequences[player_final_sequences['possession_id'].isin(index_counts[index_counts > 5].index)]
possession_index = player_final_sequences["possession_id"].unique()

/Users/morten/anaconda3/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [11]:
#find ids of those possessions that result in a goal
goal_posession_index = sequences_filtered.loc[sequences_filtered["shot_outcome"]=="Goal", "possession_id"]

Nu begynder jeg at arbejde på, at lave spillerne til nodes

In [34]:
player_final_sequences

,player_id,pass_recipient_id,possession_id,xg,timestamp,sequence
37125,20055.0,5492.0,37644403,0.069672,00:00:23.093,12
37126,5492.0,5213.0,37644403,0.069672,00:00:26.568,11
37127,5213.0,8118.0,37644403,0.069672,00:00:30.323,10
37128,8118.0,5492.0,37644403,0.069672,00:00:34.279,9
37129,5492.0,5213.0,37644403,0.069672,00:00:39.131,8
...,...,...,...,...,...,...
14306,5203.0,6379.0,3773672171,0.101797,00:47:00.677,5
14307,6379.0,22390.0,3773672171,0.101797,00:47:04.417,4
14308,22390.0,21881.0,3773672171,0.101797,00:47:05.103,3
14309,21881.0,6947.0,3773672171,0.101797,00:47:07.817,2


In [43]:
import networkx as nx

#iterate over possession ids and each row and append edges to a list for each graph and append that graph to a graphs dictionary (directed graph created with "nx.DiGraph(edges)")
#xg added as an attribute for each graph
graphs_dict = {}
for j in possession_index:
    edges = []
    for i in player_final_sequences.index:
        if j == player_final_sequences["possession_id"][i]:
            edges.append((player_final_sequences["player_id"][i], player_final_sequences["pass_recipient_id"][i]))
            if j not in graphs_dict:
                graphs_dict[j] = {"xg": player_final_sequences["xg"][i], "graph": None}
            else:
                graphs_dict[j]["xg"] = player_final_sequences["xg"][i]
                
    graph = nx.DiGraph()
    graph.add_edges_from(edges)

    # Add sequence as an edge attribute
    for i in player_final_sequences.index:
        if j == player_final_sequences["possession_id"][i]:
            graph[player_final_sequences["player_id"][i]][player_final_sequences["pass_recipient_id"][i]]['sequence'] = player_final_sequences["sequence"][i]

    graphs_dict[j]["graph"] = graph

    graph_list = [value["graph"] for value in graphs_dict.values()]

graph_list = [value["graph"] for value in graphs_dict.values()]

man kunne gøre noget med at fixere spillerene på deres positioner på banen for at visualisere det. Det gider jeg ikke at bruge tid på lige nu

In [47]:
graph_list = [value["graph"] for value in graphs_dict.values()]
edge_matrix = [list(graph.edges(data=True)) for graph in graph_list]
GRAPH_DB = graph_list  # List of graphs in the database
min_sup = 0


In [55]:
from itertools import combinations
from networkx.algorithms.isomorphism import DiGraphMatcher


def frequent_singletons(min_sup, edge_matrix):
    items_counted = {}
    edge_attributes = {}

    for edge_list in edge_matrix:
        for edge in edge_list:
            # Use only source and target nodes for counting
            edge_key = (edge[0], edge[1])
            items_counted[edge_key] = items_counted.get(edge_key, 0) + 1
            
            # Store the edge attributes
            if edge_key not in edge_attributes:
                edge_attributes[edge_key] = edge[2]

    # Filter edges that meet the min_sup
    F = [key for key, value in items_counted.items() if value >= min_sup]
    
    F_graphs = []
    for edge in F:
        g = nx.DiGraph()
        g.add_edge(edge[0], edge[1], **edge_attributes[edge])  # Add edge with original attributes
        F_graphs.append(g)
    
    return F_graphs

def generate_candidates(F, k):
    candidates = set()
    
    # Iterate over all pairs of frequent subgraphs (F)
    for g1, g2 in combinations(F, 2):

        edges_g1 = sorted(g1.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g2 = sorted(g2.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g1 = edges_g1[0]
        edges_g2 = edges_g2[0]
        
        if edges_g1[2].get('sequence') > edges_g2[2].get('sequence') or edges_g1[2].get('sequence') < edges_g2[2].get('sequence'):
            union_graph = nx.compose(g1, g2)
            
            # Ensure that the union has the correct number of edges
            if union_graph.number_of_edges() == k:  # Check edge size instead of node size
                candidates.add(union_graph)
    
    return candidates

# Count the support for each candidate in the graph database
def count_support(C, graph_db):
    F_count = {}
    for graph in graph_db:
        for candidate in C:
            GM = DiGraphMatcher(graph, candidate)
            if GM.subgraph_is_isomorphic():  # Check for subgraph isomorphism
                F_count[candidate] = F_count.get(candidate, 0) + 1
    return F_count

# Filter frequent candidates based on minimum support
def filter_frequent(F_count, min_sup):
    return [key for key, value in F_count.items() if value >= min_sup]

# Main function to run the apriori graph mining algorithm
def apriori_graph_mining(min_sup, edge_matrix, graph_db, max_k):
    frequent_total = []
    
    # Step 1: Find frequent singletons (edges)
    F = frequent_singletons(min_sup, edge_matrix)
    
    # Add initial frequent items to the total list
    frequent_total.extend(F)
    
    k = 2  # Start with size-2 subgraphs
    while k <= max_k:
        print(f"\nIteration {k}:")
        
        # Step 2: Generate candidate subgraphs of size k
        C = generate_candidates(F, k)
        
        # Step 3: Count support for each candidate in the graph database
        F_count = count_support(C, graph_db)
        
        # Step 4: Filter out frequent candidates that meet the minimum support
        F = filter_frequent(F_count, min_sup)
        
        if not F:  # If no frequent candidates are found, stop the algorithm
            print(f"No frequent subgraphs found for size {k}. Terminating.")
            break
        
        # Add frequent items to the total list
        frequent_total.extend(F)
        
        print(f"Frequent subgraphs of size {k}:")
        for subgraph in F:
            for u, v, attr in subgraph.edges(data=True):
                print(f"Edge {u} -> {v}, Attributes: {attr}")
        
        k += 1  # Move to the next size of subgraphs

    return frequent_total


graph_list = [value["graph"] for value in graphs_dict.values()]

edge_matrix = [list(graph.edges(data=True)) for graph in graph_list]
GRAPH_DB = graph_list  # List of graphs in the database
min_sup = 0


frequents = apriori_graph_mining(20, edge_matrix, GRAPH_DB, 5)
for pattern in frequents:
    print(pattern.edges())


Iteration 2:
Frequent subgraphs of size 2:
Edge 5203.0 -> 5503.0, Attributes: {'sequence': 4}
Edge 6826.0 -> 5211.0, Attributes: {'sequence': 2}
Edge 5211.0 -> 5503.0, Attributes: {'sequence': 10}
Edge 6826.0 -> 5211.0, Attributes: {'sequence': 2}
Edge 5211.0 -> 5503.0, Attributes: {'sequence': 10}
Edge 5203.0 -> 5503.0, Attributes: {'sequence': 4}

Iteration 3:
Frequent subgraphs of size 3:
Edge 5203.0 -> 5503.0, Attributes: {'sequence': 4}
Edge 6826.0 -> 5211.0, Attributes: {'sequence': 2}
Edge 5211.0 -> 5503.0, Attributes: {'sequence': 10}
Edge 5211.0 -> 5503.0, Attributes: {'sequence': 10}
Edge 6826.0 -> 5211.0, Attributes: {'sequence': 2}
Edge 5203.0 -> 5503.0, Attributes: {'sequence': 4}

Iteration 4:
No frequent subgraphs found for size 4. Terminating.
[(5211.0, 5503.0)]
[(5203.0, 5503.0)]
[(6826.0, 5211.0)]
[(5203.0, 5503.0), (6826.0, 5211.0)]
[(5211.0, 5503.0), (6826.0, 5211.0)]
[(5211.0, 5503.0), (5203.0, 5503.0)]
[(5203.0, 5503.0), (6826.0, 5211.0), (5211.0, 5503.0)]
[(5211

In [56]:
pattern

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

game_pos = 377366114

# Draw the graph using the positions
nx.draw_networkx(graphs_dict[game_pos]["graph"])

# Show the plot
plt.show()


In [ ]:
len(graphs_dict)